# AULA 5

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd

In [2]:
spark = SparkSession.builder.appName('Aula de Spark 5').enableHiveSupport().getOrCreate()

In [3]:
spark

In [4]:
results = spark.read.csv('s3a://camada-bronze-aula5/futebol', header = True)

In [5]:
results.show(5)

+----------+-------------+-------------+--------------+--------------+--------------+--------+--------------+-----------+
|      date|home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName|cityCity|countryCountry|neutralTRUE|
+----------+-------------+-------------+--------------+--------------+--------------+--------+--------------+-----------+
|1872-11-30|     Scotland|      England|             0|             0|      Friendly| Glasgow|      Scotland|      FALSE|
|1873-03-08|      England|     Scotland|             4|             2|      Friendly|  London|       England|      FALSE|
|1874-03-07|     Scotland|      England|             2|             1|      Friendly| Glasgow|      Scotland|      FALSE|
|1875-03-06|      England|     Scotland|             2|             2|      Friendly|  London|       England|      FALSE|
|1876-03-04|     Scotland|      England|             3|             0|      Friendly| Glasgow|      Scotland|      FALSE|
+----------+------------

In [6]:
results.describe().show()

+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+--------------+-----------+
|summary|      date|home_teamName|away_teamName|    home_scoreHome|    away_scoreAway| tournamentName|           cityCity|countryCountry|neutralTRUE|
+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+--------------+-----------+
|  count|     40839|        40839|        40839|             40839|             40839|          40839|              40839|         40839|      40839|
|   mean|      null|         null|         null|1.7457087587844953| 1.188104507945836|           null|               null|          null|       null|
| stddev|      null|         null|         null| 1.749144918253105|1.4051198000976632|           null|               null|          null|       null|
|    min|1872-11-30|     Abkhazia|     Abkhazia|                 0|                 0|ABCS Tournamen

In [7]:
results.printSchema()

root
 |-- date: string (nullable = true)
 |-- home_teamName: string (nullable = true)
 |-- away_teamName: string (nullable = true)
 |-- home_scoreHome: string (nullable = true)
 |-- away_scoreAway: string (nullable = true)
 |-- tournamentName: string (nullable = true)
 |-- cityCity: string (nullable = true)
 |-- countryCountry: string (nullable = true)
 |-- neutralTRUE: string (nullable = true)



In [8]:
partidas = (
    results
    .drop('neutralTrue')
    .withColumnRenamed('date','DATA')
    .withColumnRenamed('home_teamName','MANDANTE')
    .withColumnRenamed('away_teamName','VISITANTE')
    .withColumnRenamed('home_scoreHome','GOL_MANDANTE').withColumn('GOL_MANDANTE',col('GOL_MANDANTE').cast('int'))
    .withColumnRenamed('away_scoreAway','GOL_VISITANTE').withColumn('GOL_VISITANTE',col('GOL_VISITANTE').cast('int'))
    .withColumnRenamed('tournamentName','TORNEIO')
    .withColumnRenamed('cityCity','CIDADE')
    .withColumnRenamed('countryCountry','PAIS')
)

In [9]:
partidas.printSchema()

root
 |-- DATA: string (nullable = true)
 |-- MANDANTE: string (nullable = true)
 |-- VISITANTE: string (nullable = true)
 |-- GOL_MANDANTE: integer (nullable = true)
 |-- GOL_VISITANTE: integer (nullable = true)
 |-- TORNEIO: string (nullable = true)
 |-- CIDADE: string (nullable = true)
 |-- PAIS: string (nullable = true)



In [10]:
partidas_pandas = partidas.toPandas()

In [11]:
partidas_pandas

,DATA,MANDANTE,VISITANTE,GOL_MANDANTE,GOL_VISITANTE,TORNEIO,CIDADE,PAIS
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland
1,1873-03-08,England,Scotland,4,2,Friendly,London,England
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland
3,1875-03-06,England,Scotland,2,2,Friendly,London,England
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland
...,...,...,...,...,...,...,...,...
40834,2019-07-18,American Samoa,Tahiti,8,1,Pacific Games,Apia,Samoa
40835,2019-07-18,Fiji,Solomon Islands,4,4,Pacific Games,Apia,Samoa
40836,2019-07-19,Senegal,Algeria,0,1,African Cup of Nations,Cairo,Egypt
40837,2019-07-19,Tajikistan,North Korea,0,1,Intercontinental Cup,Ahmedabad,India


# Python (Pyspark)
# SQL

In [12]:
partidas.createOrReplaceTempView('v_partidas')

In [13]:
spark.sql('select * from v_partidas limit 2').show()

+----------+--------+---------+------------+-------------+--------+-------+--------+
|      DATA|MANDANTE|VISITANTE|GOL_MANDANTE|GOL_VISITANTE| TORNEIO| CIDADE|    PAIS|
+----------+--------+---------+------------+-------------+--------+-------+--------+
|1872-11-30|Scotland|  England|           0|            0|Friendly|Glasgow|Scotland|
|1873-03-08| England| Scotland|           4|            2|Friendly| London| England|
+----------+--------+---------+------------+-------------+--------+-------+--------+



In [14]:
partidas.describe().show()

+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|summary|      DATA|     MANDANTE|    VISITANTE|      GOL_MANDANTE|     GOL_VISITANTE|        TORNEIO|             CIDADE|       PAIS|
+-------+----------+-------------+-------------+------------------+------------------+---------------+-------------------+-----------+
|  count|     40839|        40839|        40839|             40839|             40839|          40839|              40839|      40839|
|   mean|      null|         null|         null|1.7457087587844953| 1.188104507945836|           null|               null|       null|
| stddev|      null|         null|         null| 1.749144918253105|1.4051198000976632|           null|               null|       null|
|    min|1872-11-30|     Abkhazia|     Abkhazia|                 0|                 0|ABCS Tournament|6th of October City|Afghanistan|
|    max|2019-07-20|Åland Islands|Åland Islands|       

In [26]:
(
    partidas \
        .select('TORNEIO') \
        .distinct() \
        .orderBy('TORNEIO') \
        .show(3,truncate=False)
)

+---------------------------+
|TORNEIO                    |
+---------------------------+
|ABCS Tournament            |
|AFC Asian Cup              |
|AFC Asian Cup qualification|
+---------------------------+
only showing top 3 rows



In [25]:
spark.sql('select distinct torneio from v_partidas order by 1').show(3,truncate=False)

+---------------------------+
|torneio                    |
+---------------------------+
|ABCS Tournament            |
|AFC Asian Cup              |
|AFC Asian Cup qualification|
+---------------------------+
only showing top 3 rows

